# A09 - Bagging

Nicolás Martínez Gutiérrez

In [129]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.stats import mode

In [130]:
data = pd.read_csv("Default.csv")
data

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880
...,...,...,...,...
9995,No,No,711.555020,52992.378910
9996,No,No,757.962918,19660.721770
9997,No,No,845.411989,58636.156980
9998,No,No,1569.009053,36669.112360


In [131]:
data["default_bin"] = (data["default"] == "Yes").astype(int)
data["student_bin"] = (data["student"] == "Yes").astype(int)
X = data[["student_bin", "balance", "income"]].values
y = data["default_bin"].values

In [132]:
logit = LogisticRegression(max_iter=3000)
logit.fit(X, y)

LogisticRegression(max_iter=3000)

In [133]:
y_logit = logit.predict(X)
p_logit = logit.predict_proba(X)[:, 1]

In [134]:
acc_logit = accuracy_score(y, y_logit)
auc_logit = roc_auc_score(y, p_logit)

In [135]:
n_arboles = 500
tam_bootstrap = 500
n_obs = X.shape[0]
columnas_totales = np.array([0, 1, 2])  
lista_arboles = []
lista_columnas = []

In [136]:
for m in range(n_arboles):
    idx_boot = np.random.choice(n_obs, size=tam_bootstrap, replace=True)
    X_boot = X[idx_boot]
    y_boot = y[idx_boot]

    cols_sel = np.random.choice(columnas_totales, size=2, replace=False)
    X_boot_sel = X_boot[:, cols_sel]

    arbol = DecisionTreeClassifier(random_state=m)
    arbol.fit(X_boot_sel, y_boot)

    lista_arboles.append(arbol)
    lista_columnas.append(cols_sel)

predicciones = np.zeros((n_obs, n_arboles))
probabilidades = np.zeros((n_obs, n_arboles))

In [137]:
for m in range(n_arboles):
    columnas = lista_columnas[m]
    modelo = lista_arboles[m]

    X_temp = X[:, columnas]

    predicciones[:, m] = modelo.predict(X_temp)
    probabilidades[:, m] = modelo.predict_proba(X_temp)[:, 1]

moda_pred = mode(predicciones, axis=1).mode.flatten()
prob_bag = probabilidades.mean(axis=1)
acc_bag = accuracy_score(y, moda_pred)
auc_bag = roc_auc_score(y, prob_bag)

In [138]:
acc_bag

0.9709

In [139]:
auc_bag

np.float64(0.9781236186015331)

In [140]:
acc_logit 

0.9732

In [141]:
auc_logit

np.float64(0.9495714810703948)

In [143]:
moda_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [144]:
import pandas as pd

tabla = pd.DataFrame({
    "Modelo": ["Bagging", "Logistic Regression"],
    "Accuracy": [0.9709, 0.9732],
    "AUC": [0.9781, 0.9496]
})

tabla

,Modelo,Accuracy,AUC
0,Bagging,0.9709,0.9781
1,Logistic Regression,0.9732,0.9496


## Conclusión

La comparación muestra que Logistic Regression obtiene un accuracy ligeramente mayor 0.9732 que el Bagging 0.9709, por lo que en términos de aciertos directos el modelo lineal tiene una mínima ventaja. Sin embargo, cuando se evalúa el AUC, el Bagging alcanza un valor más alto, 0.9781 frente a 0.9496, lo que significa que es más efectivo separando a los clientes de mayor y menor riesgo, incluso si su predicción final es menos precisa. Finalmente, al analizar la moda de las predicciones del Bagging, que resume los 500 votos de los árboles para cada observación, se observa que la mayoría de los clasificadores tienden a votar por “No default”. Esto pasa porque casi no hay clientes que caen en “default”, así que la mayoría de los árboles termina votando por la clase que más aparece en los datos, que es “No default”. Aun así, el modelo de bagging logra un AUC muy alto, lo que significa que, aunque casi siempre vote por cero, sus probabilidades internas sí distinguen bien entre clientes de bajo y alto riesgo.